# Recursive Character Text Splitter
`RecursiveCharacterTextSplitter`는  **가장 지능적인** 텍스트 분할 도구입니다.  
CharacterTextSplitter의 단일 separator 방식과 달리, **여러 separator를 순차적으로 시도**하여 가장 자연스러운 분할점을 찾습니다.  
대부분의 RAG 시스템에서 권장되는 방식입니다
### 주요 특징
1. **지능적 분할**: 여러 separator를 우선순위에 따라 순차 적용합니다
2. **자연스러운 경계**: 문단 → 문장 → 단어 순으로 자연스러운 분할을 수행합니다
3. **유연한 설정**: separator 목록을 사용자가 정의 가능합니다


In [1]:
from langchain_community.document_loaders import TextLoader

# test.txt 파일의 경로 설정
file_path = "../data/ai.txt"

# TextLoader를 사용하여 텍스트 파일 로드
loader = TextLoader(file_path, encoding='utf-8')

# 문서 로드
document = loader.load()

print(f"로드된 문서 수: {len(document)}")
print(document)

로드된 문서 수: 1
[Document(metadata={'source': '../data/ai.txt'}, page_content="머신러닝은 인공지능(AI)의 핵심적인 한 분야로, 컴퓨터 시스템이 명시적인 프로그래밍 없이 데이터로부터 자동으로 학습하고, 패턴을 인식하며, 의사결정이나 예측을 수행할 수 있도록 하는 기술입니다. 이 과정은 대규모 데이터를 알고리즘에 입력하여 시스템이 시간이 지남에 따라 스스로 성능을 조정하고 개선하도록 만드는 것을 포함합니다. 그 응용 분야는 매우 광범위하여, 헬스케어부터 금융에 이르기까지 산업 전반에 걸쳐 혁신을 주도하고 있습니다.\n머신러닝의 가장 일반적인 접근법은 지도학습입니다. 이 방법은 '레이블' 즉, 정답이 지정된 데이터셋을 사용하여 모델을 훈련시킵니다. 모델의 목표는 새로운 데이터가 주어졌을 때 그 결과값을 정확히 예측할 수 있는 내부 규칙을 학습하는 것입니다. 지도학습은 주로 분류와 회귀 문제에 사용됩니다. 분류는 이메일을 '스팸'과 '정상'으로 구분하거나, 의료 영상을 보고 종양이 '양성'인지 '악성'인지 판단하는 것처럼 이산적인 범주를 예측하는 기술입니다. 반면 회귀는 주택의 여러 특징을 기반으로 가격을 예상하거나 과거 데이터를 통해 미래 주가를 예측하는 등 연속적인 숫자 값을 예측하는 데 활용됩니다.\n지도학습과 달리, 비지도학습은 레이블이 없는 데이터를 다룹니다. 이 방식은 데이터 내에 숨겨진 구조나 패턴을 알고리즘이 스스로 탐색하고 찾아냅니다. 대표적인 예로 클러스터링이 있는데, 이는 유사한 데이터들을 하나의 그룹으로 묶는 기술로, 기업에서 고객을 구매 패턴에 따라 여러 그룹으로 나누어 맞춤형 마케팅을 하는 데 사용됩니다. 또한, 데이터의 중요한 정보는 유지하면서 변수의 개수를 줄이는 차원 축소 기술도 비지도학습에 포함되어, 복잡한 데이터를 시각화하거나 분석 효율을 높이는 데 기여합니다.\n또 다른 주요 학습 방법으로는 강화학습이 있습니다. 강화학습은 '에이전트'가 '환경'과 상호작용하며 학습하는 방식입

## 2. CharacterTextSplitter와의 비교

`RecursiveCharacterTextSplitter`의 핵심 차이점을 이해해보겠습니다.

### 주요 차이점

| 구분 | CharacterTextSplitter | RecursiveCharacterTextSplitter |
|------|----------------------|-------------------------------|
| **Separator** | 단일 separator | 여러 separator 목록 (우선순위) |
| **분할 방식** | 고정된 기준으로 분할 | 적응적으로 최적 기준 선택 |
| **자연스러움** | 기계적 분할 | 의미 단위 보존하며 분할 |
| **사용 복잡도** | 간단 | 약간 복잡하지만 더 지능적 |
| **결과 품질** | 기본적 | 더 자연스럽고 의미 있는 분할 |

### Recursive 동작 원리

1. **1단계**: 첫 번째 separator(`"\n\n"`)로 분할 시도
2. **2단계**: 청크가 너무 크면 두 번째 separator(`"\n"`)로 재분할
3. **3단계**: 여전히 크면 세 번째 separator(`". "`)로 재분할
4. **4단계**: 마지막으로 네 번째 separator(`" "`)로 단어 단위 분할
5. **최종**: 모든 separator를 시도해도 크면 강제로 문자 단위 분할


**실습** - 눈으로 확인하기위해 chunck_size를 낮게 설정.

In [2]:
from langchain_text_splitters import CharacterTextSplitter

character_splitter = CharacterTextSplitter(
    separator="\n",       # 문단 구분만 사용
    chunk_size=100,         # 200자 제한
    chunk_overlap=20
)

character_chunks = character_splitter.split_documents(document)

print(f"총 {len(character_chunks)}개 청크 생성\n")

for i, chunk in enumerate(character_chunks, 1):
    chunk_length = len(chunk.page_content)
    print(f"청크 {i} ({chunk_length}자):")
    print(f"'{chunk.page_content}'")


Created a chunk of size 245, which is longer than the specified 100
Created a chunk of size 332, which is longer than the specified 100
Created a chunk of size 278, which is longer than the specified 100
Created a chunk of size 278, which is longer than the specified 100
Created a chunk of size 299, which is longer than the specified 100
Created a chunk of size 386, which is longer than the specified 100
Created a chunk of size 231, which is longer than the specified 100
Created a chunk of size 289, which is longer than the specified 100
Created a chunk of size 430, which is longer than the specified 100


총 10개 청크 생성

청크 1 (245자):
'머신러닝은 인공지능(AI)의 핵심적인 한 분야로, 컴퓨터 시스템이 명시적인 프로그래밍 없이 데이터로부터 자동으로 학습하고, 패턴을 인식하며, 의사결정이나 예측을 수행할 수 있도록 하는 기술입니다. 이 과정은 대규모 데이터를 알고리즘에 입력하여 시스템이 시간이 지남에 따라 스스로 성능을 조정하고 개선하도록 만드는 것을 포함합니다. 그 응용 분야는 매우 광범위하여, 헬스케어부터 금융에 이르기까지 산업 전반에 걸쳐 혁신을 주도하고 있습니다.'
청크 2 (332자):
'머신러닝의 가장 일반적인 접근법은 지도학습입니다. 이 방법은 '레이블' 즉, 정답이 지정된 데이터셋을 사용하여 모델을 훈련시킵니다. 모델의 목표는 새로운 데이터가 주어졌을 때 그 결과값을 정확히 예측할 수 있는 내부 규칙을 학습하는 것입니다. 지도학습은 주로 분류와 회귀 문제에 사용됩니다. 분류는 이메일을 '스팸'과 '정상'으로 구분하거나, 의료 영상을 보고 종양이 '양성'인지 '악성'인지 판단하는 것처럼 이산적인 범주를 예측하는 기술입니다. 반면 회귀는 주택의 여러 특징을 기반으로 가격을 예상하거나 과거 데이터를 통해 미래 주가를 예측하는 등 연속적인 숫자 값을 예측하는 데 활용됩니다.'
청크 3 (278자):
'지도학습과 달리, 비지도학습은 레이블이 없는 데이터를 다룹니다. 이 방식은 데이터 내에 숨겨진 구조나 패턴을 알고리즘이 스스로 탐색하고 찾아냅니다. 대표적인 예로 클러스터링이 있는데, 이는 유사한 데이터들을 하나의 그룹으로 묶는 기술로, 기업에서 고객을 구매 패턴에 따라 여러 그룹으로 나누어 맞춤형 마케팅을 하는 데 사용됩니다. 또한, 데이터의 중요한 정보는 유지하면서 변수의 개수를 줄이는 차원 축소 기술도 비지도학습에 포함되어, 복잡한 데이터를 시각화하거나 분석 효율을 높이는 데 기여합니다.'
청크 4 (278자):
'또 다른 주요 학습 방법으로는 강화학습이 있습니다. 강화학습은 '에이전트'가 '환경'과 상호작용하며 학습하는 방식입니다. 에이

단일 문단으로 split을 했을때 청크가 너무 길어지거나 너무 많은 문장이 하나의 청크에 섞이는 문제가 발생한다.  
이럴때 `RecursiveCharacterTextSplitter`를 사용할 수 있다.

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

recursive_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " "],  # 문단 → 줄 → 문장 → 단어 순
    chunk_size=100,    
    chunk_overlap=20
)

recursive_chunks = recursive_splitter.split_documents(document)

print(f"총 {len(recursive_chunks)}개 청크 생성\n")

for i, chunk in enumerate(recursive_chunks, 1):
    chunk_length = len(chunk.page_content)
    print(f"청크 {i} ({chunk_length}자):")
    print(f"'{chunk.page_content}'")

총 47개 청크 생성

청크 1 (99자):
'머신러닝은 인공지능(AI)의 핵심적인 한 분야로, 컴퓨터 시스템이 명시적인 프로그래밍 없이 데이터로부터 자동으로 학습하고, 패턴을 인식하며, 의사결정이나 예측을 수행할 수 있도록'
청크 2 (22자):
'예측을 수행할 수 있도록 하는 기술입니다'
청크 3 (75자):
'. 이 과정은 대규모 데이터를 알고리즘에 입력하여 시스템이 시간이 지남에 따라 스스로 성능을 조정하고 개선하도록 만드는 것을 포함합니다'
청크 4 (62자):
'. 그 응용 분야는 매우 광범위하여, 헬스케어부터 금융에 이르기까지 산업 전반에 걸쳐 혁신을 주도하고 있습니다.'
청크 5 (72자):
'머신러닝의 가장 일반적인 접근법은 지도학습입니다. 이 방법은 '레이블' 즉, 정답이 지정된 데이터셋을 사용하여 모델을 훈련시킵니다'
청크 6 (89자):
'. 모델의 목표는 새로운 데이터가 주어졌을 때 그 결과값을 정확히 예측할 수 있는 내부 규칙을 학습하는 것입니다. 지도학습은 주로 분류와 회귀 문제에 사용됩니다'
청크 7 (87자):
'. 분류는 이메일을 '스팸'과 '정상'으로 구분하거나, 의료 영상을 보고 종양이 '양성'인지 '악성'인지 판단하는 것처럼 이산적인 범주를 예측하는 기술입니다'
청크 8 (84자):
'. 반면 회귀는 주택의 여러 특징을 기반으로 가격을 예상하거나 과거 데이터를 통해 미래 주가를 예측하는 등 연속적인 숫자 값을 예측하는 데 활용됩니다.'
청크 9 (81자):
'지도학습과 달리, 비지도학습은 레이블이 없는 데이터를 다룹니다. 이 방식은 데이터 내에 숨겨진 구조나 패턴을 알고리즘이 스스로 탐색하고 찾아냅니다'
청크 10 (96자):
'. 대표적인 예로 클러스터링이 있는데, 이는 유사한 데이터들을 하나의 그룹으로 묶는 기술로, 기업에서 고객을 구매 패턴에 따라 여러 그룹으로 나누어 맞춤형 마케팅을 하는 데'
청크 11 (23자):
'나누어 맞춤형 마케팅을 하는 데 사용됩니다'
청크 12 (95자):
'. 또한, 데이터의 중요한 정